### SUGGESTION ALGORITHM: 

#### DISPLAYS THE NEAREST POST OFFICE LOCATIONS NEAR THE END POINT.

- NO DATASET CLEANNING IMPLEMENTED
- CAN BE IMPROVED USING GOOGLE'S GEOCODE API


In [10]:
# Overhead Required.

import pandas as pd
import numpy as np
import requests

def get_lat_lng(address, api_key):
    url = 'https://api.opencagedata.com/geocode/v1/json'
    params = {
        'q': address,
        'key': api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            lat_lng = data['results'][0]['geometry']
            return lat_lng['lat'], lat_lng['lng']
        else:
            return None, None
    else:
        print(f"Error: {response.status_code}")
        return None, None

api_key = '909c37d5fcb1424290a6f93a4137c216'  

def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians 
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of earth in kilometers
    return c * r

def find_nearby_locations(df, target_lat, target_lon, threshold_km):
    """
    Returns nearby locations within a certain threshold (in km) from the target latitude and longitude.

    Parameters:
    df (DataFrame): DataFrame containing 'Latitude' and 'Longitude' columns.
    target_lat (float): Target latitude.
    target_lon (float): Target longitude.
    threshold_km (float): Distance threshold in kilometers.

    Returns:
    DataFrame: A DataFrame with rows within the specified distance from the target location.
    """
    distances = df.apply(lambda row: haversine(target_lat, target_lon, row['Latitude'], row['Longitude']), axis=1)
    
    # Filter DataFrame for locations within the specified threshold distance
    nearby_df = df[distances <= threshold_km].copy()
    
    # Add a column to show the calculated distances
    nearby_df['Distance_km'] = distances[distances <= threshold_km]
    
    return nearby_df


# Data Cleaning

ALL_2024 = pd.read_csv("2024_Aug.csv", encoding='unicode_escape')
ALL_2024['Latitude'] = pd.to_numeric(ALL_2024['Latitude'], errors='coerce')
ALL_2024['Longitude'] = pd.to_numeric(ALL_2024['Longitude'], errors='coerce')
VALID_2024 = ALL_2024[((ALL_2024['Latitude'].between(-90, 90)) & (ALL_2024['Longitude'].between(-180, 180)))] 

C:\Users\Admin\AppData\Local\Temp\ipykernel_2072\1358414201.py:69: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  ALL_2024 = pd.read_csv("2024_Aug.csv", encoding='unicode_escape')


In [14]:
# STEP 1 : READ THE ADDRESS AND CLEAN IT APPROIATELY

# ENTER YOUR ADDRESS. IF ADDRESS NOT FOUND ABSTRACT THE ADDRESS. MAKE IT MORE GENERAL: REMOVE SPECIFIC DETAILS. 
cus_end_point = "Amarajyothi Hospital, Kamalapur Road, Dharwad, Karnataka" 

## Extract other variables like CircleName, RegionName, DivisionName, District, StateName using an LLM


In [15]:
# Step 2 : Get Latitude and Logitude

lat, lng = get_lat_lng(cus_end_point, api_key)

if lat and lng:
    print(f"Latitude: {lat}, Longitude: {lng}")
    print(f"https://www.google.com/maps?q={lat},{lng}")
else:
    print("Address not found.")


Latitude: 15.46005, Longitude: 75.00778
https://www.google.com/maps?q=15.46005,75.00778


In [16]:
# Step 3 : Find Nearby Postoffice Delivery Hubs

tol = 10 # Kilometer Radius

Query = find_nearby_locations(VALID_2024, lat, lng, 10).sort_values(by="Distance_km").head(10)



In [17]:
Query

,CircleName,RegionName,DivisionName,OfficeName,Pincode,OfficeType,Delivery,District,StateName,Latitude,Longitude,Distance_km
153548,Karnataka Circle,North Karnataka Region,Dharwad Division,Dharwad Line Bazar S.O,580001,PO,Non Delivery,DHARWAD,KARNATAKA,15.455063,75.009044,0.570858
156392,Karnataka Circle,North Karnataka Region,Dharwad Division,Dharwad Market S.O,580001,PO,Non Delivery,DHARWAD,KARNATAKA,15.461202,75.013275,0.602718
152572,Karnataka Circle,North Karnataka Region,Dharwad Division,Dharwad K.C.Park S.O,580008,PO,Delivery,DHARWAD,KARNATAKA,15.461754,75.001708,0.677763
153768,Karnataka Circle,North Karnataka Region,Dharwad Division,Dharwad City S.O,580001,PO,Non Delivery,DHARWAD,KARNATAKA,15.460424,75.016058,0.888162
124147,Karnataka Circle,North Karnataka Region,Dharwad Division,Dharwad H.O,580001,HO,Delivery,DHARWAD,KARNATAKA,15.451833,75.006250,0.928253
152573,Karnataka Circle,North Karnataka Region,Dharwad Division,Dharwad Kutchery S.O,580001,PO,Non Delivery,DHARWAD,KARNATAKA,15.452304,75.002560,1.027069
153549,Karnataka Circle,North Karnataka Region,Dharwad Division,Dharwad M B Nagar S.O,580008,PO,Non Delivery,DHARWAD,KARNATAKA,15.469849,75.004086,1.159302
153770,Karnataka Circle,North Karnataka Region,Dharwad Division,Dharwad Narayanpur S.O,580008,PO,Non Delivery,DHARWAD,KARNATAKA,15.461481,74.995226,1.354815
38682,Karnataka Circle,North Karnataka Region,Dharwad Division,Kavalgeri B.O,580006,BO,Delivery,DHARWAD,KARNATAKA,15.470373,75.018130,1.596195
153550,Karnataka Circle,North Karnataka Region,Dharwad Division,Dharwad M J Nagar S.O,580006,PO,Delivery,DHARWAD,KARNATAKA,15.470333,75.018278,1.604116
